# Survey Responses and Map Layers

The table `responses_anonymous_pivotsrc.csv` contains every answer for every survey question as a row (except for questions with personal information.

Rows that have a value in the `fid` column have a corresponding `feature` object in the provided `geojson` files.

You'll encounter two types of `fid` values: 

* those that are **integers**&mdash;these answers have a corresponding feature in one of the survey's map layers; the `fid` and `text` in the row corresponds to `appid` and `title` in the `geojson` feature, respectively.
* those that are **strings**, in the format of `<tag>.<id>`&mdash;these were points-of-interest created by the user; the `fid` is from the Mapbox Geocoder. We *did not* keep/store the coordinates for those points, as doing so is against Mapbox's TOU. However, the `text` field contains the *exact* address or place name returned by the geocoder...which may be useful.

The table below, a slice of `responses_anonymous_pivotsrc.csv`, shows both types. The first two rows have features in the map layer that was used for question 5.

| question | fid | text |
| --- | --- | --- |
| 5 - Destinations | 18239 | CCP Monroeville | 
| 5 - Destinations | 19329 | Children's Hospital of Pittsburgh East | 
| 5 - Destinations | poi.2233383000457 | Walmart Supercenter, 100 Walmart Dr, North Versailles, Pennsylvania 15137, United States | 
| 5 - Destinations | poi.2783138891149 | ALDI, 2348 Ardmore Blvd, Pittsburgh, Pennsylvania 15221, United States | 


The table `question_layer_lookup.csv` indicates the relation between survey choices and features in a layer (also shown below):

| Question | GeoJSON |
| --- | --- |
| 3 - Area of interest | ppt_concept_areas.json |
| 4 - Communities of interest | ppt_concept_communities.json |
| 5 - Destinations | ppt_concept_areas_destinations.json |
| 6 - Corridor preference |  ppt_concept_routes.json |
| 7 - Pittsburgh areas | ppt_concept_regions_pgh.json |
| 8 - Allegheny County areas | ppt_concept_regions_ac.json |
| 9 - PAAC Routes Used | paac_routes.json |



Below we'll walk through an example of joining a summary of the results for a question to the `geojson` for visualization on a map in external GIS software like QGIS.

We're using [PETL](https://petl.readthedocs.io) in this example, but obviously feel free to use whatever you like.

In [1]:
# import some packages
import json
import petl as etl

In [2]:
# open the geojson for the areas question (#3)
with open('geojson/ppt_concept_communities.json') as fp:
    ppt_layer = json.load(fp)
    
# look at the properties for one of them:
ppt_layer['features'][0]['properties']

{'id': 2,
 'data': None,
 'appid': 150,
 'title': 'Monroeville',
 'detail': None,
 'apptype': None,
 'appareaid': 2,
 'description': None}

In [3]:
# open the results csv and get just the responses for the question
ppt_responses = etl\
    .fromcsv('responses_anonymous_pivotsrc.csv')\
    .selecteq('question', '4 - Communities of interest')\

ppt_responses

id,timestamp,question,fid,text
009a7c74-36df-485b-b9ab-4d89d73ae94c,2019-08-23T12:16:01,4 - Communities of interest,100,Duquesne
009a7c74-36df-485b-b9ab-4d89d73ae94c,2019-08-23T12:16:01,4 - Communities of interest,90,Homestead Borough
009a7c74-36df-485b-b9ab-4d89d73ae94c,2019-08-23T12:16:01,4 - Communities of interest,95,Munhall Borough
009a7c74-36df-485b-b9ab-4d89d73ae94c,2019-08-23T12:16:01,4 - Communities of interest,92,West Homestead Borough
01846a81-b601-46a5-aa51-3b1f94c78f95,2019-08-17T16:33:35,4 - Communities of interest,30,East Pittsburgh Borough


For each feature in the GeoJSON, tally up the corresponding answers in the results table. Join it back to the GeoJSON feature. In parallel, you can create a new summary table.

In [7]:
sum_tables = []

for f in ppt_layer['features']:
    
    # get the join id 
    appid = f['properties']['appid']
    
    # use it to select records from the survey responses, count them up, and get the resulting sum out
    t = etl\
        .selecteq(ppt_responses, 'fid', str(appid))\
        .aggregate(['fid', 'text'], len)\
        .rename({'value': 'count', 'fid': 'appid'})
    
    # push that table to a list
    sum_tables.append(t)
    
    # add the sum as a new property to the geojson feature
    count = list(etl.values(t, 'count'))
    if count:
        f['properties']['total'] = count[0]
    else:
        f['properties']['total'] = 0
    
summary_table = etl\
    .stack(*sum_tables)\
    .convert('count', int)\
    .convert('appid', int)

summary_table

appid,text,count
150,Monroeville,263
75,North Versailles Township,141
186,Whitaker Borough,50
10,Braddock Borough,215
30,East Pittsburgh Borough,143


Note that the feature we looked at earlier now has a `total` property:

In [8]:
ppt_layer['features'][0]['properties']

{'id': 2,
 'data': None,
 'appid': 150,
 'title': 'Monroeville',
 'detail': None,
 'apptype': None,
 'appareaid': 2,
 'description': None,
 'total': 263}

Here's another example, using `folium` to create a map in the notebook.

In [9]:
import folium

Here we're just reading the PETL summary table generated above into a Pandas dataframe, but the whole pivot table process could be done in Pandas:

In [10]:
df = etl.todataframe(summary_table)
df

,appid,text,count
0,150,Monroeville,263
1,75,North Versailles Township,141
2,186,Whitaker Borough,50
3,10,Braddock Borough,215
4,30,East Pittsburgh Borough,143
5,155,Pitcairn Borough,50
6,185,Chalfant Borough,42
7,50,Turtle Creek Borough,166
8,20,North Braddock Borough,143
9,65,Wall Borough,27


Join the GeoJson and Dataframe together and put it on a map using Folium

In [13]:
m = folium.Map(
    location=[40.4026109, -79.8340136], 
    zoom_start=11,
    tiles='Stamen Toner'
)

folium.Choropleth(
    geo_data=ppt_layer,
    data=df,
    columns=['appid', 'count'],
    key_on='feature.properties.appid',
    fill_color='BuPu',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Count',
    name='3 - Area of interest'
).add_to(m)

m